<a href="https://colab.research.google.com/github/dvsh243/mech-interp/blob/main/gemma-steer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install dependencies
try:
    import transformer_lens
except:
    %pip install einops datasets jaxtyping "sae_lens>=3.23.1" openai tabulate umap-learn hdbscan eindex-callum git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python git+https://github.com/callummcdougall/sae_vis.git@callum/v3 "transformer-lens>=2.7.0"


In [2]:
import gc
import itertools
import math
import os
import random
import sys
from collections import Counter, defaultdict
from copy import deepcopy
from dataclasses import dataclass
from functools import partial
from pathlib import Path
from typing import Any, Callable, Literal, TypeAlias

import circuitsvis as cv
import einops
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import requests
import torch as t
from datasets import load_dataset
from huggingface_hub import hf_hub_download
from IPython.display import HTML, IFrame, clear_output, display
from jaxtyping import Float, Int
from openai import OpenAI
from rich import print as rprint
from rich.table import Table
from sae_lens import (
    SAE,
    ActivationsStore,
    HookedSAETransformer,
    LanguageModelSAERunnerConfig,
    SAEConfig,
    SAETrainingRunner,
    upload_saes_to_huggingface,
)
from sae_lens.toolkit.pretrained_saes_directory import get_pretrained_saes_directory
from sae_vis import SaeVisConfig, SaeVisData, SaeVisLayoutConfig
from tabulate import tabulate
from torch import Tensor, nn
from torch.distributions.categorical import Categorical
from torch.nn import functional as F
from tqdm.auto import tqdm
from transformer_lens import ActivationCache, HookedTransformer
from transformer_lens.hook_points import HookPoint
from transformer_lens.utils import get_act_name, test_prompt, to_numpy

device = t.device("mps" if t.backends.mps.is_available() else "cuda" if t.cuda.is_available() else "cpu")

MAIN = __name__ == "__main__"

In [3]:
from sae_lens.toolkit.pretrained_saes_directory import get_pretrained_saes_directory

# TODO: Make this nicer.
df = pd.DataFrame.from_records(
    {k: v.__dict__ for k, v in get_pretrained_saes_directory().items()}
).T
df.drop(
    columns=[
        "expected_var_explained",
        "expected_l0",
        "config_overrides",
        "conversion_func",
    ],
    inplace=True,
)
df

,release,repo_id,model,saes_map,neuronpedia_id
deepseek-r1-distill-llama-8b-qresearch,deepseek-r1-distill-llama-8b-qresearch,qresearch/DeepSeek-R1-Distill-Llama-8B-SAE-l19,deepseek-ai/DeepSeek-R1-Distill-Llama-8B,{'blocks.19.hook_resid_post': 'DeepSeek-R1-Dis...,{'blocks.19.hook_resid_post': 'deepseek-r1-dis...
gemma-2-2b-res-matryoshka-dc,gemma-2-2b-res-matryoshka-dc,chanind/gemma-2-2b-batch-topk-matryoshka-saes-...,gemma-2-2b,{'blocks.0.hook_resid_post': 'standard/blocks....,"{'blocks.0.hook_resid_post': None, 'blocks.1.h..."
gemma-2-2b-res-snap-matryoshka-dc,gemma-2-2b-res-snap-matryoshka-dc,chanind/gemma-2-2b-batch-topk-matryoshka-saes-...,gemma-2-2b,{'blocks.0.hook_resid_post': 'snap/blocks.0.ho...,"{'blocks.0.hook_resid_post': None, 'blocks.1.h..."
gemma-2-9b-res-matryoshka-dc,gemma-2-9b-res-matryoshka-dc,chanind/gemma-2-9b-batch-topk-matryoshka-saes-...,gemma-2-9b,{'blocks.0.hook_resid_post': 'blocks.0.hook_re...,"{'blocks.0.hook_resid_post': None, 'blocks.1.h..."
gemma-2b-it-res-jb,gemma-2b-it-res-jb,jbloom/Gemma-2b-IT-Residual-Stream-SAEs,gemma-2b-it,{'blocks.12.hook_resid_post': 'gemma_2b_it_blo...,{'blocks.12.hook_resid_post': 'gemma-2b-it/12-...
...,...,...,...,...,...
sae_bench_gemma-2-2b_vanilla_width-2pow16_date-1109,sae_bench_gemma-2-2b_vanilla_width-2pow16_date...,canrager/saebench_gemma-2-2b_width-2pow16_date...,gemma-2-2b,{'blocks.12.hook_resid_post__trainer_0': 'gemm...,{'blocks.12.hook_resid_post__trainer_0': 'gemm...
sae_bench_pythia70m_sweep_gated_ctx128_0730,sae_bench_pythia70m_sweep_gated_ctx128_0730,canrager/lm_sae,pythia-70m-deduped,{'blocks.3.hook_resid_post__trainer_0': 'pythi...,{'blocks.3.hook_resid_post__trainer_0': 'pythi...
sae_bench_pythia70m_sweep_panneal_ctx128_0730,sae_bench_pythia70m_sweep_panneal_ctx128_0730,canrager/lm_sae,pythia-70m-deduped,{'blocks.3.hook_resid_post__trainer_16': 'pyth...,{'blocks.3.hook_resid_post__trainer_16': 'pyth...
sae_bench_pythia70m_sweep_standard_ctx128_0712,sae_bench_pythia70m_sweep_standard_ctx128_0712,canrager/lm_sae,pythia-70m-deduped,{'blocks.3.hook_resid_post__trainer_10': 'pyth...,{'blocks.3.hook_resid_post__trainer_10': 'pyth...


No charts were generated by quickchart


In [4]:
from google.colab import userdata
print( userdata.get('hf_token') )

gemma2 = HookedSAETransformer.from_pretrained("gemma-2-2b", device=device)
layer = 9

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/google/gemma-2-2b.
401 Client Error. (Request ID: Root=1-67f76ed4-23554d1f7ac500e7271d78af;9ab34353-f368-40f9-afc6-d89f55355f61)

Cannot access gated repo for url https://huggingface.co/google/gemma-2-2b/resolve/main/config.json.
Access to model google/gemma-2-2b is restricted. You must have access to it and be authenticated to access it. Please log in.

In [ ]:
block = 'blocks.6.hook_resid_post'